In [0]:
storage_account_name = "databricksdemostorage"
storage_account_key = dbutils.secrets.get("Keys", "Storage")
container = "data"

spark.conf.set(f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net", storage_account_key)

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType

schema = StructType([
  StructField("Vendor", StringType(), True),
  StructField("Model", StringType(), True),
  StructField("CycleTime", DoubleType(), True),
  StructField("MinMainMemory", DoubleType(), True),
  StructField("MaxMainMemory", DoubleType(), True),
  StructField("Cache", DoubleType(), True),
  StructField("MinChannels", DoubleType(), True),
  StructField("MaxChannels", DoubleType(), True),
  StructField("PublishedPerf", DoubleType(), True),
  StructField("RelativePerf", DoubleType(), True)
])

In [0]:
data = spark.read.option("header", "true").option("delimeter", ",").schema(schema).csv(f"wasbs://{container}@{storage_account_name}.blob.core.windows.net/machine.data")

data.show()

+---------+--------+---------+-------------+-------------+-----+-----------+-----------+-------------+------------+
 Vendor| Model|CycleTime|MinMainMemory|MaxMainMemory|Cache|MinChannels|MaxChannels|PublishedPerf|RelativePerf|
+---------+--------+---------+-------------+-------------+-----+-----------+-----------+-------------+------------+
 amdahl| 470v/7| 29.0| 8000.0| 32000.0| 32.0| 8.0| 32.0| 269.0| 253.0|
 amdahl| 470v/7a| 29.0| 8000.0| 32000.0| 32.0| 8.0| 32.0| 220.0| 253.0|
 amdahl| 470v/7b| 29.0| 8000.0| 32000.0| 32.0| 8.0| 32.0| 172.0| 253.0|
 amdahl| 470v/7c| 29.0| 8000.0| 16000.0| 32.0| 8.0| 16.0| 132.0| 132.0|
 amdahl| 470v/b| 26.0| 8000.0| 32000.0| 64.0| 8.0| 32.0| 318.0| 290.0|
 amdahl|580-5840| 23.0| 16000.0| 32000.0| 64.0| 16.0| 32.0| 367.0| 381.0|
 amdahl|580-5850| 23.0| 16000.0| 32000.0| 64.0| 16.0| 32.0| 489.0| 381.0|
 amdahl|580-5860| 23.0| 16000.0| 64000.0| 64.0| 16.0| 32.0| 636.0| 749.0|
 amdahl|580-5880| 23.0| 32000.0| 64000.0|128.0| 32.0| 64.0| 1144.0| 1238.0|
 apollo| dn320| 400.0| 1000.0| 3000.0| 0.0| 1.0| 2.0| 38.0| 23.0|
 apollo| dn420| 400.0| 512.0| 3500.0| 4.0| 1.0| 6.0| 40.0| 24.0|
 basf| 7/65| 60.0| 2000.0| 8000.0| 65.0| 1.0| 8.0| 92.0| 70.0|
 basf| 7/68| 50.0| 4000.0| 16000.0| 65.0| 1.0| 8.0| 138.0| 117.0|
 bti| 5000| 350.0| 64.0| 64.0| 0.0| 1.0| 4.0| 10.0| 15.0|
 bti| 8000| 200.0| 512.0| 16000.0| 0.0| 4.0| 32.0| 35.0| 64.0|
burroughs| b1955| 167.0| 524.0| 2000.0| 8.0| 4.0| 15.0| 19.0| 23.0|
burroughs| b2900| 143.0| 512.0| 5000.0| 0.0| 7.0| 32.0| 28.0| 29.0|
burroughs| b2925| 143.0| 1000.0| 2000.0| 0.0| 5.0| 16.0| 31.0| 22.0|
burroughs| b4955| 110.0| 5000.0| 5000.0|142.0| 8.0| 64.0| 120.0| 124.0|
burroughs| b5900| 143.0| 1500.0| 6300.0| 0.0| 5.0| 32.0| 30.0| 35.0|
+---------+--------+---------+-------------+-------------+-----+-----------+-----------+-------------+------------+
only showing top 20 rows

In [0]:
(train_data, test_data) = data.randomSplit([0.8, 0.2])

In [0]:
print(train_data.count())
print(test_data.count())

167
41

In [0]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator

In [0]:
vectors = VectorAssembler(inputCols=['CycleTime', 'MinMainMemory', 'MaxMainMemory', 'Cache', 'MinChannels', 'MaxChannels'], outputCol="features")

vector_data = vectors.transform(train_data)

In [0]:
vector_data.show()

+---------+--------+---------+-------------+-------------+-----+-----------+-----------+-------------+------------+--------------------+
 Vendor| Model|CycleTime|MinMainMemory|MaxMainMemory|Cache|MinChannels|MaxChannels|PublishedPerf|RelativePerf| features|
+---------+--------+---------+-------------+-------------+-----+-----------+-----------+-------------+------------+--------------------+
 amdahl| 470v/7| 29.0| 8000.0| 32000.0| 32.0| 8.0| 32.0| 269.0| 253.0|[29.0,8000.0,3200...|
 amdahl| 470v/7a| 29.0| 8000.0| 32000.0| 32.0| 8.0| 32.0| 220.0| 253.0|[29.0,8000.0,3200...|
 amdahl| 470v/7b| 29.0| 8000.0| 32000.0| 32.0| 8.0| 32.0| 172.0| 253.0|[29.0,8000.0,3200...|
 amdahl| 470v/7c| 29.0| 8000.0| 16000.0| 32.0| 8.0| 16.0| 132.0| 132.0|[29.0,8000.0,1600...|
 amdahl| 470v/b| 26.0| 8000.0| 32000.0| 64.0| 8.0| 32.0| 318.0| 290.0|[26.0,8000.0,3200...|
 amdahl|580-5840| 23.0| 16000.0| 32000.0| 64.0| 16.0| 32.0| 367.0| 381.0|[23.0,16000.0,320...|
 amdahl|580-5850| 23.0| 16000.0| 32000.0| 64.0| 16.0| 32.0| 489.0| 381.0|[23.0,16000.0,320...|
 amdahl|580-5880| 23.0| 32000.0| 64000.0|128.0| 32.0| 64.0| 1144.0| 1238.0|[23.0,32000.0,640...|
 apollo| dn320| 400.0| 1000.0| 3000.0| 0.0| 1.0| 2.0| 38.0| 23.0|[400.0,1000.0,300...|
 apollo| dn420| 400.0| 512.0| 3500.0| 4.0| 1.0| 6.0| 40.0| 24.0|[400.0,512.0,3500...|
 basf| 7/68| 50.0| 4000.0| 16000.0| 65.0| 1.0| 8.0| 138.0| 117.0|[50.0,4000.0,1600...|
 bti| 5000| 350.0| 64.0| 64.0| 0.0| 1.0| 4.0| 10.0| 15.0|[350.0,64.0,64.0,...|
 bti| 8000| 200.0| 512.0| 16000.0| 0.0| 4.0| 32.0| 35.0| 64.0|[200.0,512.0,1600...|
burroughs| b1955| 167.0| 524.0| 2000.0| 8.0| 4.0| 15.0| 19.0| 23.0|[167.0,524.0,2000...|
burroughs| b2900| 143.0| 512.0| 5000.0| 0.0| 7.0| 32.0| 28.0| 29.0|[143.0,512.0,5000...|
burroughs| b2925| 143.0| 1000.0| 2000.0| 0.0| 5.0| 16.0| 31.0| 22.0|[143.0,1000.0,200...|
burroughs| b4955| 110.0| 5000.0| 5000.0|142.0| 8.0| 64.0| 120.0| 124.0|[110.0,5000.0,500...|
burroughs| b5900| 143.0| 1500.0| 6300.0| 0.0| 5.0| 32.0| 30.0| 35.0|[143.0,1500.0,630...|
burroughs| b6900| 143.0| 2300.0| 6200.0| 0.0| 6.0| 64.0| 61.0| 40.0|[143.0,2300.0,620...|
 c.r.d|68/10-80| 320.0| 128.0| 6000.0| 0.0| 1.0| 12.0| 23.0| 28.0|[320.0,128.0,6000...|
+---------+--------+---------+-------------+-------------+-----+-----------+-----------+-------------+------------+--------------------+
only showing top 20 rows

In [0]:
features_data = vector_data.select(["features", "PublishedPerf"])

features_data.show()

+--------------------+-------------+
 features|PublishedPerf|
+--------------------+-------------+
[29.0,8000.0,3200...| 269.0|
[29.0,8000.0,3200...| 220.0|
[29.0,8000.0,3200...| 172.0|
[23.0,16000.0,320...| 367.0|
[23.0,16000.0,320...| 489.0|
[23.0,16000.0,640...| 636.0|
[23.0,32000.0,640...| 1144.0|
[400.0,1000.0,300...| 38.0|
[400.0,512.0,3500...| 40.0|
[60.0,2000.0,8000...| 92.0|
[50.0,4000.0,1600...| 138.0|
[350.0,64.0,64.0,...| 10.0|
[200.0,512.0,1600...| 35.0|
[167.0,524.0,2000...| 19.0|
[143.0,512.0,5000...| 28.0|
[143.0,1000.0,200...| 31.0|
[110.0,5000.0,500...| 120.0|
[143.0,1500.0,630...| 30.0|
[143.0,3100.0,620...| 33.0|
[143.0,2300.0,620...| 61.0|
+--------------------+-------------+
only showing top 20 rows

In [0]:
lr = LinearRegression(labelCol="PublishedPerf", featuresCol="features")

In [0]:
model = lr.fit(features_data)

In [0]:
summary = model.summary

print("R^2", summary.r2)

R^2 0.907080674424948

In [0]:
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="PublishedPerf", metricName="r2")

In [0]:
vector_test = vectors.transform(test_data)

In [0]:
features_test = vector_test.select(["features", "PublishedPerf"])

In [0]:
test_transform = model.transform(features_test)

test_transform.show()

+--------------------+-------------+------------------+
 features|PublishedPerf| prediction|
+--------------------+-------------+------------------+
[29.0,8000.0,1600...| 132.0| 225.236784789747|
[26.0,8000.0,3200...| 318.0|280.99322131708396|
[50.0,2620.0,1048...| 63.0|119.14210263447198|
[56.0,5240.0,2097...| 106.0| 196.7310559934032|
[64.0,5240.0,2097...| 208.0|196.93537687474017|
[50.0,1000.0,4000...| 29.0| 16.2533389496358|
[133.0,1000.0,800...| 72.0| 36.13488993430222|
[810.0,1000.0,500...| 20.0| 33.57328396481017|
[140.0,1000.0,160...| 138.0| 51.57310185375148|
[75.0,2000.0,1600...| 144.0| 96.61323875364235|
[75.0,2000.0,1600...| 259.0|132.72339176144706|
[300.0,768.0,4500...| 45.0|14.423213490666388|
[300.0,192.0,768....| 36.0|16.875975551692942|
[330.0,1000.0,300...| 16.0|20.424811696720408|
[300.0,1000.0,400...| 38.0|29.695777712391685|
[330.0,1000.0,400...| 22.0|26.668648938825303|
[140.0,2000.0,400...| 22.0| 38.41800473764438|
[140.0,2000.0,320...| 189.0|117.99624976453889|
[105.0,256.0,2000...| 26.0|1.2063810420912304|
[175.0,256.0,2000...| 22.0| 2.820290156142505|
+--------------------+-------------+------------------+
only showing top 20 rows

In [0]:
evaluator.evaluate(test_transform)

Out[24]: 0.5614136494199689